# Identifying Covid-19 using Chest X-ray with Deep Transfer Learning

In [32]:
# imports

# tensorflow is used as backend for keras
import tensorflow as tf

# imports from keras
# we will need these as we are using trnsfer learning and we will have to train our own last layer(s)
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model, Sequential
# importing pre-trained modeland a fuction to pre-process our image
from keras.applications.densenet import DenseNet201, preprocess_input
# for image data augmentation
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# usual imports
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

### Datasets
Covid-19 X-ray:
https://github.com/ieee8023/covid-chestxray-dataset  
Pneumonia and Normal X-ray:
https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

In [33]:
train_path = "Datasets/train"
test_path = "Datasets/test"

In [34]:
folders = glob("Datasets/Train/*")
print(folders)

['Datasets/Train\\NORMAL', 'Datasets/Train\\PNEUMONIA']


In [35]:
LABELS = ["NORMAL", "PNEUMONIA"]
# LABELS = ["COVID", "NORMAL"]

### Downloading the pretrained DenseNet201 model with weights trained for ImageNet:

More Info.: https://keras.io/api/applications/

In [36]:
# we need to resize our image to this size
IMAGE_SIZE = [224, 224]

In [37]:
# we will not include last layer include_top=False
densenet = DenseNet201(include_top=False, weights='imagenet', input_shape=IMAGE_SIZE + [3], classes=1000)

In [38]:
# we don't need to train any layer
for layer in densenet.layers:
    layer.trainable=False

### Model Construction

In [39]:
x = Flatten()(densenet.output)
# x = Dense(1000, activation="relu")(x)
prediction = Dense(len(folders), activation="softmax")(x)

In [40]:
# creating a model object
model = Model(inputs=densenet.input, outputs=prediction)

In [41]:
# model structure
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
_______________________________________________________________________________________

conv5_block13_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block13_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block13_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block13_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block13_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block13_1_relu[0][0]       
__________________________________________________________________________________________________
conv5_block13_concat (Concatena (None, 7, 7, 1312)   0           conv5_block12_concat[0][0]       
                                                                 conv5_block13_2_conv[0][0]       
__________________________________________________________________________________________________
conv5_block14_0_bn (BatchNormal (None, 7, 7, 1312)   5248        conv5_block13_concat[0][0]       
__________

In [42]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### Image Preprocessing

In [43]:
# data augmentation
train_datagen = ImageDataGenerator(rescale=1/255, shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1/255)

In [44]:
training_set = train_datagen.flow_from_directory(train_path,
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode="categorical")

Found 5216 images belonging to 2 classes.


In [45]:
test_set = test_datagen.flow_from_directory(test_path, target_size=(224, 224),
                                           batch_size=32,
                                           class_mode="categorical")

Found 624 images belonging to 2 classes.


In [46]:
r = model.fit_generator(training_set,
                       validation_data=test_set,
                       epochs=5,
                       steps_per_epoch=len(training_set),
                       validation_steps=len(test_set))

Epoch 1/5
  1/163 [..............................] - ETA: 0s - loss: 1.1807 - accuracy: 0.3125

KeyboardInterrupt: 

In [26]:
model.save("new_model.h5")

In [27]:
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="val loss")
plt.legend()
plt.show()
#plt.save("lossval_loss")

NameError: name 'r' is not defined

In [28]:
plt.plot(r.history['accuracy'], label="train acc")
plt.plot(r.history['val_accuracy'], label="val acc")
plt.legend()
plt.show()

NameError: name 'r' is not defined

In [30]:
def prepare(filepath):
    IMG_SIZE = 224
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(1, IMG_SIZE, IMG_SIZE, 3)

In [31]:
models = tf.keras.models.load_model("new_model.h5")
#prediction = model.predict([prepare('p1.jpg')])
prediction = model.predict(["COVID-19.jpg"])
print(prediction)
#print(LABELS[int(prediction[0][0])])

y_classes = prediction.argmax(axis=1)
print(y_classes)
print(LABELS[int(y_classes[0])])

ValueError: in user code:

    C:\Users\ujwal\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\ujwal\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ujwal\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ujwal\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ujwal\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ujwal\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1445 run_step 